In [810]:
import math
import inspect
import numpy as np
import matplotlib.pyplot as plt

def Conv2D(kernel_depth, kernel_size = 3, strides = 1, average = True):
    def pad(X, pad):
        batch_size, input_row, input_col, input_channels = X.shape
        
        padded_image = np.zeros((batch_size, input_row + (2 * pad), input_col + (2 * pad), input_channels))
        padded_image[:, pad:pad + input_row, pad: pad + input_col, :] = X
        
        return padded_image
    
    def convolute_slide(x_slice, W, b):
        return np.sum(np.multiply(x_slice, W)) + b
    
    def conv_forward(X, W, b = 0):
        # Retrieve dimensions from input shape
        batch_size, input_row, input_col, num_channels = X.shape
        
        # Retrieve dimensions from W's shape
        kernel_size, kernel_size, num_channels, output_channels = W.shape
        
        # Calculate the required padding size
        pad_size = int((kernel_size - 1) / 2)
        
        # Calculate output dimensions
        output_height = int(((input_row - kernel_size + 2 * pad_size) / strides)) + 1
        output_width = int(((input_col - kernel_size + 2 * pad_size) / strides)) + 1
        
        # Create output dimensions         
        output = np.zeros((batch_size, output_height, output_width, output_channels))
        
        # Pad input dimensions    
        padded_input = pad(X, pad_size)
        
        # Loop over number of training examples        
        for i in range(batch_size):
            padded_input_i = padded_input[i]
            
            for row in range(output_height):
                row_start = row * strides
                row_end = row_start + kernel_size
                
                for col in range(output_width):
                    col_start = col * strides
                    col_end = col_start + kernel_size
                    
                    for c in range(output_channels):
                        padded_input_slice = padded_input_i[row_start: row_end, col_start: col_end]
                        weights = W[..., c]
                        biases = b[..., c]
                        output[i, row, col, c] = convolute_slide(padded_input_slice, weights, biases)
        
        return output
    
    def apply(inputs):
        num_channels = inputs.shape[3]
        W = np.random.randn(kernel_size, kernel_size, num_channels, kernel_depth)
        b = np.random.randn(1, 1, 1, kernel_depth)
        return conv_forward(inputs, W, b)
    
    return apply

In [839]:
import functools

def compose(*functions):
    def compose2(f, g):
        return lambda x: f(g(x))
    return functools.reduce(compose2, functions, lambda x: x)

X = np.array([
    [[1,1,1,1], [3,3,3,3], [5,5,5,5]],
    [[2,2,2,2], [4,4,4,4], [6,6,6,6]],
    [[2,2,2,2], [4,4,8,8], [3,3,4,4]],
    [[3,3,3,3], [5,5,5,5], [7,7,7,7]],
    [[5,5,5,5], [8,8,8,8], [4,3,3,4]],
    [[5,5,5,5], [7,7,7,7], [9,9,9,9]],
    [[1,1,1,1], [8,7,7,8], [9,9,9,9]],
    [[1,2,2,1], [8,8,8,8], [4,4,4,4]],
], dtype=np.float32)

X = X[..., np.newaxis]
y = np.array([[1, 1, 0, 1, 0, 1, 0, 0]])

In [1340]:
class Flatten(Layer):
    def build(self):
        self.add_weight()
    
class Dense(Layer):
    def __init__(self, layer_size, activation = Activation.relu, **kwargs):
        super(Dense, self).__init__(**kwargs)
        self.layer_size = layer_size
        self.activation = activation
        
    def build(self):
        self.add_weight()

class Layer(object):
    def __init__(self, **kwargs):
        self._trainable_weights = []
        
    def add_weight(self):
        pass
    
class Network(object):
    def __init__(self, *args, **kwargs):
        print(args)
        pass
    
class Model(Network):
    def fit(self):
        pass

class Sequential(Model):
    def __init__(self, layers=None, name=None):
        super(Sequential, self).__init__(name=name)
        
        if layers:
            for layer in layers:
                self.add(layer)
        
    def add(self, layer):
        layer.build()

In [1341]:
model = Sequential([
    Dense(16, input_shape=(3, 4))
])

AttributeError: 'tuple' object has no attribute 'name'

In [1315]:
# I: (12, 8), W1: (12, 16) -> (16, 8)
# I: (16, 8), W2: (16, 32) -> (32, 8)